In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import dictys

In [ ]:
from utils_custom import *

In [ ]:
# Define file paths 
data_file = '/ocean/projects/cis240075p/asachan/datasets/B_Cell/multiome_1st_donor_UPMC_aggr/dictys_outs/actb1_added/output/dynamic.h5'
output_folder = '/ocean/projects/cis240075p/asachan/datasets/B_Cell/multiome_1st_donor_UPMC_aggr/dictys_outs/actb1_added/output'
tf_pairs_file = '/ocean/projects/cis240075p/asachan/datasets/B_Cell/multiome_1st_donor_UPMC_aggr/other_files/combinatorial_control/SLIDE_LF_enriched_pair_GC-1_PB-2.csv'

In [ ]:
# Load data
dictys_dynamic_object = dictys.net.dynamic_network.from_file(data_file)

In [ ]:
# Assign window indices for each branch between 0 to 60 (these are relative to S0)
pb_window_indices = list(range(30, 46)) + [2]
abc_window_indices = [1] + list(range(29, 3, -1)) + [0]
gc_window_indices = list(range(46, 61)) + [3]
pb_with_actb_indices = abc_window_indices + pb_window_indices
gc_with_actb_indices = abc_window_indices + gc_window_indices

In [ ]:
# get TF pairs from combinatorial control
tf_pairs_df = pd.read_csv(tf_pairs_file)
tf_pairs_and_targets = tf_pairs_df[['TF', 'common']]
display(tf_pairs_and_targets)

In [ ]:
all_tfs = []
for pair_str in tf_pairs_and_targets['TF']:
    tf1, tf2 = eval(pair_str)  # Convert string representation to actual tuple
    all_tfs.extend([tf1, tf2])  # Add both TFs to the list

# Get unique TFs using set
unique_tfs = sorted(set(all_tfs))

print(f"Number of unique TFs: {len(unique_tfs)}")
print("Unique TFs:", unique_tfs)

In [ ]:
# get unique targets by opening up the {} where all targets are stored and appending to a set
unique_targets = set()
for target_set in tf_pairs_and_targets['common']:
    unique_targets.update(eval(target_set))
unique_targets = sorted(unique_targets)
print(f"Number of unique targets: {len(unique_targets)}")
print("Unique targets:", unique_targets)


In [ ]:
cc_tf_target_pairs = create_tf_target_pairs(dictys_dynamic_object, tf_pairs_and_targets)
display(cc_tf_target_pairs)

In [ ]:
# Get unique TFs and targets from the pairs
tfs = sorted(set(pair[0] for pair in cc_tf_target_pairs))
targets = sorted(set(pair[1] for pair in cc_tf_target_pairs))

# Create DataFrame with TFs as rows and targets as columns
cc_tf_target_pairs_df = pd.DataFrame(0, index=tfs, columns=targets)

# Fill in the DataFrame with 1s for existing TF-target pairs
for tf, target in cc_tf_target_pairs:
    cc_tf_target_pairs_df.loc[tf, target] = 1

display(cc_tf_target_pairs_df)

In [ ]:
# now get the tf indices for the tfs in cc_tf_target_pairs
tf_indices_cc, _ = get_tf_indices(dictys_dynamic_object, tfs)
display(len(tf_indices_cc))

In [ ]:
# get the gene indices for the targets in cc_tf_target_pairs
gene_indices_cc = get_target_gene_indices(dictys_dynamic_object, targets)
display(len(gene_indices_cc))
#display(gene_indices_cc)

In [ ]:
# get weights across windows
weights_tf_target_pb = get_weights_for_tf_target_pairs(dictys_dynamic_object, tf_indices_cc, gene_indices_cc, pb_window_indices)
weights_tf_target_gc = get_weights_for_tf_target_pairs(dictys_dynamic_object, tf_indices_cc, gene_indices_cc, gc_window_indices)

In [ ]:
display(weights_tf_target_pb.shape)

In [ ]:
display(np.max(weights_tf_target_pb))
display(np.min(weights_tf_target_pb))
display(np.max(weights_tf_target_gc))
display(np.min(weights_tf_target_gc))


In [ ]:
# Example usage
mask_gc = filter_tf_target_pairs(weights_tf_target_gc, 
                                      sparsity_threshold=0.99,  
                                      weight_percentile=90.00)
filtered_pairs_gc = get_filtered_pairs(mask_gc, tfs, targets, cc_tf_target_pairs)

# Get number of pairs that pass the filter
print(f"Number of filtered pairs: {len(filtered_pairs_gc)}")
display(filtered_pairs_gc)

In [ ]:
mask_pb = filter_tf_target_pairs(weights_tf_target_pb, 
                                      sparsity_threshold=0.9,  
                                      weight_percentile=99.00)   
filtered_pairs_pb = get_filtered_pairs(mask_pb, tfs, targets, cc_tf_target_pairs)

# Get number of pairs that pass the filter
print(f"Number of filtered pairs: {len(filtered_pairs_pb)}")
display(filtered_pairs_pb)

In [ ]:
# create a list of tuples with pairs of pax5 coming from both branches, accounting for duplicates
filtered_pairs_all = list(set(filtered_pairs_pb + filtered_pairs_gc))
display(len(filtered_pairs_all))

In [ ]:
fig, ax, cmap = fig_regulation_heatmap(
    network=dictys_dynamic_object,
    start=0,
    stop=3,
    regulations=filtered_pairs_all,
    num=100,  # Number of points to interpolate
    dist=0.0005,  # Gaussian kernel smoothing distance/length scale between cells. Larger value means stronger smoothing.
    cmap='RdBu',  # Red indicates higher values, Blue indicates lower values
    figsize=(2, 0.2),  # Reduced row height to 0.15
)

In [ ]:
fig, ax, cmap = fig_regulation_heatmap(
    network=dictys_dynamic_object,
    start=0,
    stop=2,
    regulations=filtered_pairs_all,
    num=100,  # Number of points to interpolate
    dist=0.0005,  # Gaussian kernel smoothing distance/length scale between cells. Larger value means stronger smoothing.
    cmap='RdBu',  # Red indicates higher values, Blue indicates lower values
    figsize=(2, 0.2),  # Reduced row height to 0.15
)

In [ ]:
# Example usage:
fig, ax, cmap = fig_expression_gradient_heatmap(
    network=dictys_dynamic_object,
    start=0,
    stop=2,
    regulations=filtered_pairs_all,
    num=100,
    dist=0.0005,
    cmap='RdYlGn', # Red = negative gradient, Yellow = neutral, Green = positive gradient
    figsize=(2, 0.2)
)

In [ ]:
# Example usage:
fig, ax, cmap = fig_expression_gradient_heatmap(
    network=dictys_dynamic_object,
    start=0,
    stop=3,
    regulations=filtered_pairs_all,
    num=100,
    dist=0.0005,
    cmap='RdYlGn', # Red = negative gradient, Yellow = neutral, Green = positive gradient
    figsize=(2, 0.2)
)